In [ ]:
!pip install wandb
!pip install transformers
!pip install torch
!pip install sentencepiece
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 3.5 MB/s 
     |████████████████████████████████| 166 kB 72.3 MB/s 
     |████████████████████████████████| 182 kB 83.5 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 166 kB 94.0 MB/s 
     |████████████████████████████████| 162 kB 93.3 MB/s 
     |████████████████████████████████| 162 kB 92.5 MB/s 
     |████████████████████████████████| 158 kB 76.3 MB/s 
     |████████████████████████████████| 157 kB 73.1 MB/s 
     |████████████████████████████████| 157 kB 100.8 MB/s 
     |████████████████████████████████| 157 kB 97.7 MB/s 
     |████████████████████████████████| 157 kB 77.6 MB/s 
     |████████████████████████████████| 157 kB 83.1 MB/s 
     |████████████████████████████████| 157 kB 78.8 MB/s 
     |████████████████████████████████| 157 kB 6.8 MB/s 
     |████████████████████████████

In [ ]:
%matplotlib inline

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import transformers
from datasets import load_dataset

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import glob
import os
import wandb
import random, os
import matplotlib.pyplot as plt
import numpy as np
from transformers import AdamW
import torch
from transformers import get_cosine_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, AutoConfig
import re
from torch.nn import Module
from collections import Counter, defaultdict
from tqdm import tqdm
from torch import nn
import sys
import gc
from transformers import DataCollatorWithPadding


# From this Gist: https://gist.github.com/ihoromi4/b681a9088f348942b01711f251e5f964
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
   

In [ ]:
CFG = {
    "seed": [42, 12, 48, 55, 83],
    "model_name": "bert-base-uncased",
    "max_length": 512,
    "lr": 2e-5, 
    "batch_size": 32,
    "epochs": 20,
    "num_warmup_steps": 0.0,
    "dataset": "mrpc",
    "type": "+ CLS",

    "patience": 6,

    "grad_accum": 1,
    "layer_start": 0,
    "pooler": None,

    "weight_decay": 0.3,
    "grad_norm": 1000,
    "optimizer": "AdamW",
    "scheduler": "linear",
}
CFG["tokenizer"] = AutoTokenizer.from_pretrained(CFG["model_name"])

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
data = load_dataset("glue", CFG["dataset"])

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(CFG["model_name"])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# cola = cola.map(lambda data: tokenizer(data["sentence"], padding=True, max_length = CFG["max_length"], truncation = True, return_token_type_ids = False), batched = True, remove_columns = ["sentence", "idx"], num_proc = 8)

#sst2
data = data.map(lambda data: tokenizer(data["sentence1"], data["sentence2"], padding=True, max_length = CFG["max_length"], truncation = True, return_token_type_ids = False), batched = True, remove_columns = ["sentence1", "idx", "sentence2"], num_proc = 8)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#5:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

#7:   0%|          | 0/1 [00:00<?, ?ba/s]

#6:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
train = data["train"]

In [ ]:
val = data["validation"]

In [ ]:
test = data["test"]

In [ ]:
train

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 3668
})

In [ ]:
train.with_format("torch", device = device)
val.with_format("torch", device = device)
test.with_format("torch", device = device)

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 1725
})

In [ ]:
#PLEASE DON'T USE MY API KEY. I'M TRUSTING YOU.

os.environ["XRT_TPU_CONFIG"] = "tpu_worker;0;10.0.0.2:8470"
os.environ['WANDB_CONSOLE'] = 'off'
os.environ['WANDB_NOTEBOOK_NAME'] = 'Deep Supervision Research - BERT + CLS.ipynb'
os.environ['WANDB_API_KEY'] = "d01c72067c0b354a28976e02d73dc7448c5499f2"
%env "WANDB_API_KEY" "d01c72067c0b354a28976e02d73dc7448c5499f2"

env: "WANDB_API_KEY"="d01c72067c0b354a28976e02d73dc7448c5499f2"


In [ ]:
wandb.login()

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find Deep Supervision Research - BERT + CLS.ipynb.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


True

In [ ]:
class WeightedLayerPooling(torch.nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, all_hidden_states):
        all_layer_embedding = all_hidden_states[self.layer_start:, :, :, :]
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()
        return weighted_average
    

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
class AccuracyTracker(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.num_correct = 0
        self.total = 0
        self.accuracy = 0

    def update(self, val, n=1):
        self.num_correct += val
        self.total += n
        self.accuracy = self.num_correct / self.total

In [ ]:
#OBVIOUSLY, CHANGE THIS AS YOU NEED. USE SELF.LOG FOR ALL IMPORTANT METRICS
class Model(nn.Module):
    def __init__(self, config, vocab_length, data_loader_len):
        super(Model, self).__init__()
        self.config = config
        self.vocab_length = vocab_length
        self.base_model = AutoModel.from_pretrained(self.config['model_name'], output_hidden_states = True)  
        self.base_model.resize_token_embeddings(vocab_length)

        if config["pooler"] == "weighted":
            self.pooler = WeightedLayerPooling(self.base_model.config.num_hidden_layers, layer_start = self.config["layer_start"])  
            self._init_weights(self.pooler.layer_weights)

        self.fc = nn.Linear(self.base_model.config.hidden_size, 1)

        self._init_weights(self.fc)
        self.data_loader_len = data_loader_len

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.base_model.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.base_model.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        elif isinstance(module, nn.parameter.Parameter):
            module.data.normal_(mean=0.0, std=self.base_model.config.initializer_range)
        else:
            print(f"Module of type {type(module)} cannot be initialized")

    def feature(self, inputs):

        if self.config["pooler"] == "weighted":
            input_ids, attention_mask = inputs["input_ids"], inputs["attention_mask"]
        
            x = self.base_model(input_ids = input_ids, attention_mask = attention_mask)["hidden_states"]

            x = torch.stack(x)
            cls_embeddings = self.pooler(x)[:, 0]

            return cls_embeddings
            
        else:
            input_ids, attention_mask = inputs["input_ids"], inputs["attention_mask"]
        
            x = self.base_model(input_ids = input_ids, attention_mask = attention_mask)["last_hidden_state"]

            return x[:, 0, :]

            

    def forward(self, inputs):
        
        features = self.feature(inputs)

        return self.fc(features)

# Packaging All The Above Functions into a Dataset

In [ ]:
#CHANGE AS NEEDED. MOST OF THE TIME, PYTORCH'S DEFAULT COLLATOR IS ENOUGH.
class DataModule():

    def __init__(self, config, train, val, test, collate_fn):
        self.config = config
        self.train, self.val, self.test = train, val, test
        self.collate_fn = collate_fn

        
    def train_dataloader(self):
        train_loader = DataLoader(self.train, batch_size=self.config["batch_size"], shuffle = True, collate_fn = self.collate_fn, pin_memory=True, num_workers = 8)
        # train_loader = DataLoader(self.train, batch_size=self.config["batch_size"], shuffle = True, collate_fn = self.collate_fn)
        return train_loader

    def val_dataloader(self):
        val_loader = DataLoader(self.val, batch_size = self.config["batch_size"], collate_fn = self.collate_fn, pin_memory=True, num_workers = 8)
        # val_loader = DataLoader(self.val, batch_size = self.config["batch_size"], collate_fn = self.collate_fn)
        return val_loader

    def test_dataloader(self):
        test_loader = DataLoader(self.test, batch_size = self.config["batch_size"], collate_fn = self.collate_fn, pin_memory=True,  num_workers = 8)

In [ ]:
def configure_optimizers(config, model):
    
        if config["optimizer"] == "AdamW":
            optimizer = AdamW(model.parameters(), weight_decay = config["weight_decay"], lr=config["lr"], correct_bias = True)
            
        else:
            optimizer = AdamW(model.parameters(), weight_decay = config["weight_decay"], lr=config["lr"], correct_bias = True)

        if config["scheduler"] == "cosine":
            scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = config["num_warmup_steps"], num_training_steps = model.data_loader_len * config["epochs"] // config["grad_accum"])

        elif config['scheduler'] == "one_cycle":
            scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr = config["lr"], pct_start = config["pct_start"], total_steps = model.data_loader_len * config["epochs"] // config["grad_accum"])
            
        elif config['scheduler'] == "linear":
            scheduler = transformers.get_linear_schedule_with_warmup(optimizer, num_warmup_steps = model.data_loader_len * config["epochs"] // config["grad_accum"] * 0.1, num_training_steps = model.data_loader_len * config["epochs"] // config["grad_accum"])
            
        else:
            scheduler = None
            
        
        return optimizer, scheduler

In [ ]:
def train_fn(config, train_loader, model, criterion, optimizer, epoch, scheduler, device):
    model.train()

    losses = AverageMeter()
    train_accuracy = AccuracyTracker()

    pbar = tqdm(train_loader, desc = f"Training Loop Epoch: {epoch}")

    scaler = torch.cuda.amp.GradScaler()

    latest_avg = None
        
    grad_norm = 0.0

    latest_acc = 0.0

    for batch_idx, batch in enumerate(pbar):

        labels = batch.pop("labels")

        inputs = batch

        for k, v in inputs.items():
            inputs[k] = v.to(device)

        labels = labels.to(device).to(torch.float16)

        batch_size = labels.size(0)

        #First Train
        
        with torch.cuda.amp.autocast():
            y_hat = model(inputs).reshape(-1)

            train_loss = criterion(y_hat, labels)

            scaled_loss = train_loss / config["grad_accum"]
        
        losses.update(train_loss.item(), batch_size)


        probs = torch.sigmoid(y_hat)

        num_correct = torch.sum((probs > 0.5).to(int) == labels)
        train_accuracy.update(num_correct, batch_size)
        
        scaler.scale(scaled_loss).backward()
        

        if ((batch_idx + 1) % config["grad_accum"] == 0) or (batch_idx + 1 == model.data_loader_len):

            scaler.unscale_(optimizer)

            # Since the gradients of optimizer's assigned params are unscaled, clips as usual:
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), config["grad_norm"])

            scaler.step(optimizer)
            scaler.update()

            optimizer.zero_grad()

            if not scheduler is None:

                scheduler.step()

                for i, lr in enumerate(scheduler.get_last_lr()):
                    wandb.log({f"Layer {i} Learning Rate": lr})

            latest_avg = f"{losses.avg:.4f}"

            latest_acc = f"{train_accuracy.accuracy:.4f}"

            # wandb.log({f"Training Loss_step": losses.val})

        text = f"Epoch: {epoch} | Training_accuracy: {latest_acc} | Training Loss_avg: {latest_avg} | Training Loss_step: {losses.val:.4f} | Learning Rate: {scheduler.get_last_lr()[0]:.4f} | Grad: {grad_norm:.4f}" if not scheduler is None else f"Epoch: {epoch} | Training Loss_avg: {latest_avg} | Training Loss_step: {losses.val:.4f} | Grad: {grad_norm:.4f}"

        pbar.set_postfix_str(text)

        pbar.refresh()

        wandb.log({f"Training Loss_step": losses.val})

        wandb.log({f"Training Accuracy Step": train_accuracy.accuracy})

    wandb.log({f"Training Loss Epoch": losses.avg})
    wandb.log({f"Training Accuracy Epoch": train_accuracy.accuracy})

    return losses.avg

In [ ]:
def valid_fn(valid_loader, model, criterion, device, epoch):
    model.eval()
    
    losses = AverageMeter()
    accuracy = AccuracyTracker()

    #A MANUAL LOOP IS NEEDED HERE SINCE TRAINER FUNCTIONS DON'T GIVE YOU ACCESS TO MODEL PREDICTIONS

    pbar = tqdm(valid_loader, desc = f"Validation Loop Epoch: {epoch}")
    for batch_idx, batch in enumerate(pbar):
        
        labels = batch.pop("labels")

        inputs = batch

        for k, v in inputs.items():
            inputs[k] = v.to(device)

        labels = labels.to(device).to(torch.float16)

        batch_size = labels.size(0)

        model = model.to(device)
        
        with torch.no_grad():
            preds = model(inputs).reshape(-1)

        val_loss = criterion(preds, labels).item()

        losses.update(val_loss, batch_size)
        
        probs = torch.sigmoid(preds)

        num_correct = torch.sum((probs > 0.5).to(int) == labels)
        accuracy.update(num_correct, batch_size)

        # pbar.set_postfix_str(f"Epoch: {epoch} | Validation Loss_avg: {losses.avg:.4f} | Validation Loss_step: {losses.val:.4f}")
        pbar.set_postfix_str(f"Epoch: {epoch} | Validation Loss_avg: {losses.avg:.4f} | Validation_accuracy_step: {accuracy.accuracy}")

    wandb.log({f"Validation Loss Epoch": losses.avg})
    wandb.log({f"Validation Accuracy Epoch": accuracy.accuracy})

    return losses.avg, accuracy.accuracy

In [ ]:
def test_fn(test_loader, model, criterion, device, checkpoint, class_names):
    with torch.no_grad():
        losses = AverageMeter()
        accuracy = AccuracyTracker()

        # PUT YOUR CUSTOM GRAPHS HERE
        
        # LOAD BEST MODEL CHECKPOINT

        saved = torch.load(checkpoint)
        model.load_state_dict(saved["model_state_dict"])
        model.eval()

        val_loss = torch.tensor([0]).to(device).to(torch.float16)

        #A MANUAL LOOP IS NEEDED HERE SINCE TRAINER FUNCTIONS DON'T GIVE YOU ACCESS TO MODEL PREDICTIONS

        pbar = tqdm(test_loader, desc = f"Getting Test Predictions")
        for batch_idx, batch in enumerate(pbar):
            labels = batch.pop("labels")

            inputs = batch

            for k, v in inputs.items():
                inputs[k] = v.to(device)

            labels = labels.to(device).to(torch.float16)

            batch_size = labels.size(0)

            model = model.to(device)
            
            y_hat = model(inputs)

            val_loss = criterion(y_hat.flatten(), labels)

            probs = torch.sigmoid(y_hat)

            num_correct = torch.sum((probs.flatten() > 0.5).to(int) == labels)

            losses.update(val_loss, batch_size)

            accuracy.update(num_correct, batch_size)

            #LOGGING SPECIFIC THINGS HERE #############################

    
        #LOGGING OOF PERFORMANCE
        wandb.log({f"Validation Accuracy": accuracy.accuracy})

        print(f"Validation Accuracy: {accuracy.accuracy}")

        return losses.avg

In [ ]:
class ModelTracker():
    def __init__(self, patience, base_path, model, path, optimizer, scheduler, mode = "maximize", metric_name = "accuracy"):
        self.patience = patience
        self.mode = mode
        self.missed = 0
        self.path = path
        self.model = model
        self.base_path = base_path
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.metric = float("-inf") if self.mode == "maximize" else float("inf")
        self.metric_name = metric_name

    def update(self, value, epoch):
        if self.mode == "maximize":
            if value > self.metric:
                print(f"Validation {self.metric_name} rose from {self.metric:.4f} to {value:.4f} on epoch {epoch}")
                self.metric = value
                
                torch.save({
                    "epoch": epoch, 
                    "model_state_dict": self.model.state_dict(), 
                    "optimizer_state_dict": self.optimizer.state_dict(),
                    "accuracy": self.metric,
                    "scheduler": self.scheduler.state_dict()
                }, f"{self.base_path}/{self.path}")

                print(f"Saved to model to {self.base_path}/{self.path}!")

                self.missed = 0

            else:
                print(f"Validation {self.metric_name} fell from {self.metric:.4f} to {value:.4f} on epoch {epoch}")
                print(f"Model did not improve on epoch {epoch}")
                self.missed += 1
        else:
            if value < self.metric:
                print(f"Validation {self.metric_name} fell from {self.metric:.4f} to {value:.4f} on epoch {epoch}")
                self.metric = value
                
                torch.save({
                    "epoch": epoch, 
                    "model_state_dict": self.model.state_dict(), 
                    "optimizer_state_dict": self.optimizer.state_dict(),
                    "loss": self.metric,
                    "scheduler": self.scheduler.state_dict()
                }, f"{self.base_path}/{self.path}")

                self.missed = 0

                print(f"Saved to model to {self.base_path}/{self.path}!")

            else:
                print(f"Validation {self.metric_name} rose from {self.metric:.4f} to {value:.4f} on epoch {epoch}")
                print(f"Model did not improve on epoch {epoch}")
                self.missed += 1

    def get_full_path(self):
        return f"{self.base_path}/{self.path}"
        
    def check_improvement(self):
        return (self.missed < self.patience if self.mode == "maximize" else self.missed > self.patience) and (self.missed < self.patience)

In [ ]:
def train_loop(train, val, test, data_collator, config, device, weights=None, base_path = "./"):
    for seed in config["seed"]:
        seed_everything(seed)
        classes = ["negative", "positive"]

        wandb.init(project="Auxillary BERT Research", entity = "goggins", group = config["dataset"], config = config, job_type = f"{config['model_name']} {config['type']}", save_code = True, reinit = True, name = f"Seed {seed}")

        criterion = torch.nn.BCEWithLogitsLoss()

        validation_criterion = torch.nn.BCEWithLogitsLoss()

        dataset = DataModule(config, train, val, test, data_collator)

        train_loader = dataset.train_dataloader()

        val_loader = dataset.val_dataloader()

        model = Model(config, len(config["tokenizer"]), len(train_loader))

        model = model.to(device)


        optimizer, scheduler = configure_optimizers(config, model)

        tracker = ModelTracker(config["patience"], base_path, model, f"seed-{seed}.pt", optimizer, scheduler)

        for epoch in range(config["epochs"]):

            train_loss = train_fn(config, train_loader, model, criterion, optimizer, epoch, scheduler, device)

            val_loss, val_accuracy = valid_fn(val_loader, model, validation_criterion, device, epoch)

            tracker.update(val_accuracy, epoch)

            if not tracker.check_improvement():
                print(f"Stopping the model at epoch {epoch} since the model did not improve!")
                break


        checkpoint = tracker.get_full_path()

        test_fn(val_loader, model, validation_criterion, device, checkpoint, classes)

        del dataset, model

        gc.collect()

        torch.cuda.empty_cache()


In [ ]:
device = torch.device("cuda")

In [ ]:
%pdb

Automatic pdb calling has been turned ON


In [ ]:
train_loop(train, val, test, data_collator, CFG, device)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a

Validation accuracy rose from -inf to 0.7034 on epoch 0
Saved to model to .//seed-42.pt!


Training Loop Epoch: 1:   0%|          | 0/115 [00:00<?, ?it/s]You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast 

...

Validation accuracy fell from 0.8725 to 0.8652 on epoch 13
Model did not improve on epoch 13


Training Loop Epoch: 14:   0%|          | 0/115 [00:00<?, ?it/s]You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast

Validation accuracy fell from 0.8725 to 0.8652 on epoch 14
Model did not improve on epoch 14
Stopping the model at epoch 14 since the model did not improve!


Getting Test Predictions:   0%|          | 0/13 [00:00<?, ?it/s]You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast

Validation Accuracy: 0.8725490570068359
